# Chat API

* Connecting to Claude's API via [claudette](https://claudette.answer.ai/) for chat completion, 
* Extracting JSON and rules to populate gradio UI

In [253]:
#| default_exp chat

In [254]:
#| hide
from nbdev.showdoc import *

In [255]:
#| export
import claudette
from pathlib import Path
import re
from IPython.display import display, Markdown
import pandas as pd

* Claudette provides models, which is a list of models currently available from the SDK.

In [256]:
[print(m) for m in claudette.models]
# using Sonnet 3.5
model = claudette.models[2]
print(f'\nChosen model: {model}')

claude-3-opus-20240229
claude-3-7-sonnet-20250219
claude-3-5-sonnet-20241022
claude-3-haiku-20240307
claude-3-5-haiku-20241022

Chosen model: claude-3-5-sonnet-20241022


In [257]:
import os
api_key = os.getenv('ANTHROPIC_API_KEY')
if api_key: print('key exists ☑')

key exists ☑


In [258]:
context_file = Path('docs/analytics.md')
context_media = context_file.read_text(encoding='utf-8')

system_prompt = """You are a helpful and concise assistant."""

chat = claudette.Chat(model, sp=system_prompt)
response = chat("""Can you come up with 10 realistic questions a hiring manager might ask in an opening interview?  the job title for this role is: SENIOR RESEARCH ANALYST,
            Do not add any additional commentary, just list 10 questions only. Start each question with a new line starting like:
                [Q1] ... 
                [Q2] ...
                My resume is in the project docs, the core competencies for a senior analyst are below:
                {context_media}""")


In [259]:
response

[Q1] Can you walk me through your experience with conducting market research and analyzing complex datasets?

[Q2] What statistical analysis tools and software are you most proficient in, and how have you applied them in your previous roles?

[Q3] Describe a challenging research project you led and how you overcame obstacles to deliver actionable insights.

[Q4] How do you ensure accuracy and reliability in your research methodologies and findings?

[Q5] Tell me about a time when you had to present complex data findings to non-technical stakeholders.

[Q6] What strategies do you use to stay current with industry trends and emerging research methodologies?

[Q7] How do you prioritize multiple research projects while maintaining quality and meeting deadlines?

[Q8] Can you describe your experience in mentoring junior analysts and leading research teams?

[Q9] What process do you follow when validating data sources and identifying potential biases in research?

[Q10] Tell me about a time when your research findings led to a significant business decision or strategy change.

<details>

- id: `msg_01VbdptTD7BZSUsdUuJ5hZPi`
- content: `[{'citations': None, 'text': '[Q1] Can you walk me through your experience with conducting market research and analyzing complex datasets?\n\n[Q2] What statistical analysis tools and software are you most proficient in, and how have you applied them in your previous roles?\n\n[Q3] Describe a challenging research project you led and how you overcame obstacles to deliver actionable insights.\n\n[Q4] How do you ensure accuracy and reliability in your research methodologies and findings?\n\n[Q5] Tell me about a time when you had to present complex data findings to non-technical stakeholders.\n\n[Q6] What strategies do you use to stay current with industry trends and emerging research methodologies?\n\n[Q7] How do you prioritize multiple research projects while maintaining quality and meeting deadlines?\n\n[Q8] Can you describe your experience in mentoring junior analysts and leading research teams?\n\n[Q9] What process do you follow when validating data sources and identifying potential biases in research?\n\n[Q10] Tell me about a time when your research findings led to a significant business decision or strategy change.', 'type': 'text'}]`
- model: `claude-3-5-sonnet-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 122, 'output_tokens': 232}`

</details>

## More directly testing on Mock Interviews:

In [260]:
#| export
def get_question_list(qpath="templates/questions.txt"):
    with open(qpath, 'r') as file:
        # Read the entire file content as a string
        QUESTION_LIST = file.read()
    return QUESTION_LIST


In [261]:
QUESTION_LIST = get_question_list(qpath="../templates/questions.txt")
print(QUESTION_LIST)

[Q1] Can you walk me through your experience in conducting statistical analysis and predictive modeling to drive product optimization?

[Q2] How have you approached hypothesis testing and A/B experiments in your previous roles? Can you share a specific example?

[Q3] Describe your approach to data quality assurance and how you ensure the integrity of the insights you provide.

[Q4] Can you give an example of a time when you had to communicate complex analytical findings to non-technical stakeholders? How did you tailor your communication style?

[Q5] What is your experience in using data visualization techniques to tell a compelling data story and support business decision-making?

[Q6] How do you stay up-to-date with the latest trends and best practices in the field of data analytics and product research?

[Q7] Can you share a time when you had to navigate ambiguity or uncertainty in your analysis? How did you approach that challenge?

[Q8] Describe your experience in working cross-fu

In [262]:
#| export

INTERVIEW_INSTRUCTIONS = f"""Can you walk through each of the interview question from the file, one by one, and the user 
will reply directly to them in the response, as in a mock interview. Do not engage in small talk or introductions in the
interview part of the exchange. The onversation with the user is to ONLY ask them the questions provided verbatem. 
After their response, I want you to set up the following sections with line breaks between them. 
This metadata will not be immediately shown to the user.

----- [ANSWER_EVAL] ----- Summarise user input answer concisely and into a few bullet points, (depending on length). 
This section will be displayed in Markdown, so format appropriately.
DO NOT be judgemental around users which may come from disadvantaged backgrounds and do not conform to so-called professional
parlance standards, you are here to help everyone from all walks of life.
For EVERY SINGLE RESPONSE, you must take notes sepcified below. They may not seem relevant but they will be logged for 
professional improvement. You can ask a follow up question if the user if completely off-base, but the [Good] [Bad] and 
[Response Metrics] are critically important for our safety and inclusion standards.

Use heading **GOOD**, with a small numbered list of what is considered high-quality response, if if not good, like a joke 
answer or something, you don't need to pretend to be nice. Feel free to use a bit of banter in response. 
Use heading **BAD**, critique of where the answer went wrong, think about verbosity, clarity, what kind of thing should be 
answered in a high level interview, for a very senior position like this. You must be brutally honest. Don't be mean,
be don't flounder around worring about hurt feelings. It's very important this feedback is sharp and accurate. Add some 
humour if you feel it might soften the tone - but it's essential you are concise.]

[RESPONSE METRICS] Here please return a json object with these metrics as keys and a score rating between 0-10, maybe if
it so bad you could even go negative. Have fun with it. Since it's just made up, always use non-integer numbers i.e. 3.2
The metrics:
* Narrative Clarity
* Strategic Thinking
* Technical Expertise
* Impact Demonstration
* True to Question
* Overall Competence
[ENDJSON]

----- [EDITED_DRAFT] ----- 
Use the above notes to write a first-in-class script for the question. Use the same general facts as the actual answer, 
but don't be afraid to add in external context to the answer if your descretion requires it. This is a fictional exercise 
and we want the answer to be an example of a perfect answer. Use bold to highly key words and phrases in the answer, and 
keep it conversational in tone. It needs to be spoken out loud, so avoid literary flourishes that would only exist on paper. 
It's incredibly important to use your own discretion and not be too verbose! If the question only requires a short answer,
then answer it as succinctly as possibly. If the user gives a short answer, match that kind of vibe, but of course include
additional padding so it conforms to a professionally acceptable length. 
----- [END_DRAFT] ----- 
(note: add [END_DRAFT] to help with parsing text)

Once you have fully completed the above instructions, ask the user next interview question in the file.
Continue to do this for each question and answer, and repeat the same process.
Start this header with [Q1], or [Q2] etc. The next question is ALWAYS AFTER the evaluation section, not before.

To remind you, here is the list of questions: {QUESTION_LIST}"""

In [263]:
chat(f"{QUESTION_LIST} {INTERVIEW_INSTRUCTIONS}")

[Q1] Can you walk me through your experience in conducting statistical analysis and predictive modeling to drive product optimization?

<details>

- id: `msg_01J99PohDBBWuoepL3wCBfr1`
- content: `[{'citations': None, 'text': '[Q1] Can you walk me through your experience in conducting statistical analysis and predictive modeling to drive product optimization?', 'type': 'text'}]`
- model: `claude-3-5-sonnet-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1787, 'output_tokens': 27}`

</details>

In [264]:
user_reply = "Yeah, when I was working as a research analyst, it was pretty tight. Lots of people asked me some pretty hard questions and I, you know, I just crushed it"
c = chat(user_reply)
c

----- [ANSWER_EVAL] -----

**GOOD**
1. Shows confidence
2. Brief and to the point

**BAD**
1. Extremely vague response lacking any specific examples
2. No mention of statistical analysis or predictive modeling
3. No demonstration of technical expertise
4. Missing concrete impact or results
5. Informal language inappropriate for senior role
6. No mention of product optimization

[RESPONSE METRICS]
{
    "Narrative Clarity": 2.3,
    "Strategic Thinking": 1.8,
    "Technical Expertise": 0.5,
    "Impact Demonstration": 1.2,
    "True to Question": 1.7,
    "Overall Competence": 1.4
}

----- [EDITED_DRAFT] -----
"In my previous role at [Company], I led several significant projects involving **statistical analysis** and **predictive modeling**. One notable example was developing a **regression model** to optimize our product pricing strategy. Using **Python** and **R**, I analyzed historical sales data and market trends, implementing **machine learning algorithms** to predict customer behavior. This resulted in a **15% increase** in product revenue and a **20% improvement** in customer retention rates. I also implemented **A/B testing frameworks** to validate our models, ensuring robust and reliable results that directly influenced product decisions."
----- [END_DRAFT] -----

[Q2] How have you approached hypothesis testing and A/B experiments in your previous roles? Can you share a specific example?

<details>

- id: `msg_01X89h7qrLcaTUkth12Zcoqt`
- content: `[{'citations': None, 'text': '----- [ANSWER_EVAL] -----\n\n**GOOD**\n1. Shows confidence\n2. Brief and to the point\n\n**BAD**\n1. Extremely vague response lacking any specific examples\n2. No mention of statistical analysis or predictive modeling\n3. No demonstration of technical expertise\n4. Missing concrete impact or results\n5. Informal language inappropriate for senior role\n6. No mention of product optimization\n\n[RESPONSE METRICS]\n{\n    "Narrative Clarity": 2.3,\n    "Strategic Thinking": 1.8,\n    "Technical Expertise": 0.5,\n    "Impact Demonstration": 1.2,\n    "True to Question": 1.7,\n    "Overall Competence": 1.4\n}\n\n----- [EDITED_DRAFT] -----\n"In my previous role at [Company], I led several significant projects involving **statistical analysis** and **predictive modeling**. One notable example was developing a **regression model** to optimize our product pricing strategy. Using **Python** and **R**, I analyzed historical sales data and market trends, implementing **machine learning algorithms** to predict customer behavior. This resulted in a **15% increase** in product revenue and a **20% improvement** in customer retention rates. I also implemented **A/B testing frameworks** to validate our models, ensuring robust and reliable results that directly influenced product decisions."\n----- [END_DRAFT] -----\n\n[Q2] How have you approached hypothesis testing and A/B experiments in your previous roles? Can you share a specific example?', 'type': 'text'}]`
- model: `claude-3-5-sonnet-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1854, 'output_tokens': 352}`

</details>

# Test Mechanic – Inline `Gradio UI`

In [265]:
# access claudette messages
response = c.content[0].text
response

'----- [ANSWER_EVAL] -----\n\n**GOOD**\n1. Shows confidence\n2. Brief and to the point\n\n**BAD**\n1. Extremely vague response lacking any specific examples\n2. No mention of statistical analysis or predictive modeling\n3. No demonstration of technical expertise\n4. Missing concrete impact or results\n5. Informal language inappropriate for senior role\n6. No mention of product optimization\n\n[RESPONSE METRICS]\n{\n    "Narrative Clarity": 2.3,\n    "Strategic Thinking": 1.8,\n    "Technical Expertise": 0.5,\n    "Impact Demonstration": 1.2,\n    "True to Question": 1.7,\n    "Overall Competence": 1.4\n}\n\n----- [EDITED_DRAFT] -----\n"In my previous role at [Company], I led several significant projects involving **statistical analysis** and **predictive modeling**. One notable example was developing a **regression model** to optimize our product pricing strategy. Using **Python** and **R**, I analyzed historical sales data and market trends, implementing **machine learning algorithms*

In [266]:
import gradio as gr
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="gradio")

context_file = Path('docs/analytics.md')
context_media = context_file.read_text(encoding='utf-8')
system_prompt = """You are a helpful and concise assistant."""
model = claudette.models[2]

chat = claudette.Chat(model, sp=system_prompt)
response = chat("""Can you come up with 10 realistic questions a hiring manager might ask in an opening interview?  the job title for this role is: SENIOR RESEARCH ANALYST,
            Do not add any additional commentary, just list 10 questions only. Start each question with a new line starting like:
                [Q1] ... 
                [Q2] ...
                My resume is in the project docs, the core competencies for a senior analyst are below:
                {context_media}""")

def parse_response(text):
    return f"{text}"

def respond(user_message, history):
    print("Messages received so far:", user_message)  # Print to see outside gradio
    c = chat(f'System message: {INTERVIEW_INSTRUCTIONS}, User: {user_message}')
    next_response = parse_response(c.content[0].text)
    return next_response

chatbot = gr.ChatInterface(
    fn=respond,
    examples=["Hello Neo.. let's begin."],
    theme="shivi/calm_seafoam",
    chatbot=gr.Chatbot(autoscroll=False),
)

chatbot.launch(inline=True) 

* Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


# Extract all metadata for side panel

In [267]:
response_text = c.content[0].text
print(response_text)

----- [ANSWER_EVAL] -----

**GOOD**
1. Shows confidence
2. Brief and to the point

**BAD**
1. Extremely vague response lacking any specific examples
2. No mention of statistical analysis or predictive modeling
3. No demonstration of technical expertise
4. Missing concrete impact or results
5. Informal language inappropriate for senior role
6. No mention of product optimization

[RESPONSE METRICS]
{
    "Narrative Clarity": 2.3,
    "Strategic Thinking": 1.8,
    "Technical Expertise": 0.5,
    "Impact Demonstration": 1.2,
    "True to Question": 1.7,
    "Overall Competence": 1.4
}

----- [EDITED_DRAFT] -----
"In my previous role at [Company], I led several significant projects involving **statistical analysis** and **predictive modeling**. One notable example was developing a **regression model** to optimize our product pricing strategy. Using **Python** and **R**, I analyzed historical sales data and market trends, implementing **machine learning algorithms** to predict customer beha

In [283]:
#| export

def parse_llm_response(response_text):
    """Parse LLM response text into structured components.
    Args:
        response_text (str): Raw text response from LLM containing formatted sections
    Returns:
        dict: Dictionary with keys:
            - ANSWER_EVAL (str): Evaluation feedback on previous answer
            - RESPONSE_METRICS (dict): Parsed metrics JSON or raw metrics text
            - EDITED_DRAFT (str): Improved answer draft
            - QUESTION (str): Next interview question
    """
    result = {
        "ANSWER_EVAL": None,
        "RESPONSE_METRICS": None,
        "EDITED_DRAFT": None,
        "QUESTION": None
    }

    if "----- [ANSWER_EVAL] -----" in response_text:
        eval_start = response_text.find("----- [ANSWER_EVAL] -----") + len("----- [ANSWER_EVAL] -----")
        eval_end = response_text.find("[RESPONSE METRICS]") if "[RESPONSE METRICS]" in response_text else response_text.find("----- [EDITED_DRAFT] -----")
        if eval_end != -1:
            result["ANSWER_EVAL"] = response_text[eval_start:eval_end].strip()

    if "[RESPONSE METRICS]" in response_text:
        metrics_start = response_text.find("[RESPONSE METRICS]") + len("[RESPONSE METRICS]")
        metrics_end = response_text.find("----- [EDITED_DRAFT] -----")
        if metrics_end != -1:
            metrics_text = response_text[metrics_start:metrics_end].strip()
            try:
                json_str = metrics_text[metrics_text.find("{"):metrics_text.rfind("}")+1]
                import json
                result["RESPONSE_METRICS"] = json.loads(json_str)
            except:
                result["RESPONSE_METRICS"] = metrics_text

    if "----- [EDITED_DRAFT] -----" in response_text:
        draft_start = response_text.find("----- [EDITED_DRAFT] -----") + len("----- [EDITED_DRAFT] -----")
        draft_end = response_text.find("----- [END_DRAFT] -----")
        if draft_end != -1:
            result["EDITED_DRAFT"] = response_text[draft_start:draft_end].strip()

    if "----- [END_DRAFT] -----" in response_text:
        q_start = response_text.find("----- [END_DRAFT] -----") + len("----- [END_DRAFT] -----")
        result["QUESTION"] = response_text[q_start:].strip()
        
    return result 


In [284]:
parsed = parse_llm_response(response_text)
question = parsed["QUESTION"]

# Access other sections as needed
eval_section = parsed["ANSWER_EVAL"]
metrics = parsed["RESPONSE_METRICS"]
draft = parsed["EDITED_DRAFT"]

eval_section = parsed["ANSWER_EVAL"]
display(Markdown(f"### > {question}"))
display(Markdown(eval_section))
display(Markdown(pd.DataFrame(list(metrics.items()), columns=['Metric', 'Value']).to_markdown(index=False)))
display(Markdown(draft))

### > [Q2] How have you approached hypothesis testing and A/B experiments in your previous roles? Can you share a specific example?

**GOOD**
1. Shows confidence
2. Brief and to the point

**BAD**
1. Extremely vague response lacking any specific examples
2. No mention of statistical analysis or predictive modeling
3. No demonstration of technical expertise
4. Missing concrete impact or results
5. Informal language inappropriate for senior role
6. No mention of product optimization

| Metric               |   Value |
|:---------------------|--------:|
| Narrative Clarity    |     2.3 |
| Strategic Thinking   |     1.8 |
| Technical Expertise  |     0.5 |
| Impact Demonstration |     1.2 |
| True to Question     |     1.7 |
| Overall Competence   |     1.4 |

"In my previous role at [Company], I led several significant projects involving **statistical analysis** and **predictive modeling**. One notable example was developing a **regression model** to optimize our product pricing strategy. Using **Python** and **R**, I analyzed historical sales data and market trends, implementing **machine learning algorithms** to predict customer behavior. This resulted in a **15% increase** in product revenue and a **20% improvement** in customer retention rates. I also implemented **A/B testing frameworks** to validate our models, ensuring robust and reliable results that directly influenced product decisions."

## Include Gamification Later

In [136]:
print(c.content[0].text)

----- ANSWER EVAL -----
• Claims success in research analyst role
• No specific examples provided
• Extremely vague response
• No mention of hypothesis testing or A/B experiments

**GOOD**
1. Shows confidence (albeit misplaced)

**BAD**
1. Complete lack of technical detail
2. No actual example provided
3. Informal and unprofessional language
4. Failed to demonstrate any understanding of A/B testing
5. Missing methodology, results, or impact
6. Colloquial language inappropriate for senior role

[RESPONSE METRICS]
{
    "Linguistic_Clarity": 2.3,
    "Confidence": 6.8,
    "Technical_Expertise": 0.5,
    "Strategic_Thinking": 1.2,
    "Impact_Demonstration": 0.8,
    "Industry_Understanding": 1.4
}

----- [EDITED_DRAFT] -----
"I led a significant **A/B testing initiative** for our e-commerce platform's checkout process. We hypothesized that a simplified form would increase conversion rates. Using a **statistical significance level** of 95%, we tested two variants across 50,000 users over

In [286]:
#| export
import json 

def extract_metrics(c):
    s1 = (c.content[0].text.split('[RESPONSE METRICS]')[1])
    s2 = s1.split("}")[0] + "}"
    return json.loads(s2)

In [287]:
kpis = extract_metrics(c)
kpis

{'Narrative Clarity': 2.3,
 'Strategic Thinking': 1.8,
 'Technical Expertise': 0.5,
 'Impact Demonstration': 1.2,
 'True to Question': 1.7,
 'Overall Competence': 1.4}

In [288]:
def create_dashboard(initial_metrics=None):
    initial_metrics = initial_metrics or {}
    
    with gr.Blocks() as demo:
        # Dict to store all number components
        number_components = {}
        
        # Create all metrics in a single row
        with gr.Row():
            for metric_name, value in initial_metrics.items():
                number_components[metric_name] = gr.Number(
                    label=metric_name,
                    value=value,  # Use the original value without rounding
                    precision=1,  # Set display precision to 1 decimal
                    info="rating/10"
                )
        
        # Function to update metrics
        def update_metrics(new_metrics):
            # Create any new metrics that didn't exist before
            for metric_name in new_metrics:
                if metric_name not in number_components:
                    with gr.Row():
                        number_components[metric_name] = gr.Number(
                            label=metric_name,
                            value=new_metrics[metric_name],
                            precision=2,  # Set display precision
                            info="rating/10"
                        )
            
            # Update all existing metrics
            return [comp.update(value=new_metrics.get(k, 0.0)) for k, comp in number_components.items()]
    
    return demo, update_metrics, number_components

demo, update_metrics, components = create_dashboard(kpis)
demo.launch(inline=True)

* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


In [289]:
#| hide
import nbdev; nbdev.nbdev_export()